In [1]:
%%time
%matplotlib inline
from new_import import *

CPU times: user 12.9 s, sys: 4.31 s, total: 17.2 s
Wall time: 10.8 s


In [2]:
%%time
# Cấu hình Daskgateway
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()

# Cấu hình truy cập dịch vụ S3
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

client

An existing cluster was found. Connecting to: easihub.9f2f42f40c134601a68634913d891637
CPU times: user 736 ms, sys: 28.1 ms, total: 764 ms
Wall time: 979 ms


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.asia.easi-eo.solutions/services/dask-gateway/clusters/easihub.9f2f42f40c134601a68634913d891637/status,


LOAD VH, VV

In [3]:
## cấu hình dữ liệu train và vh vv file
train_path = "train/ST_training data_updated_1130points.shp"  # đường dẫn shp file train
name_vh = "vh-0922_0923-full_ST.tif"
name_vv = "vv-0922_0923-full_ST.tif"


train = load_train_data(train_path)

In [4]:
%%time
## tải về dữ liệu sen1
import os
if not os.path.exists(name_vh):
    !aws s3 cp s3://easi-asia-dc-data/staging/ctu/sentinel-1/vh-0922_0923-full_ST.tif vh-0922_0923-full_ST.tif
if not os.path.exists(name_vv):
    !aws s3 cp s3://easi-asia-dc-data/staging/ctu/sentinel-1/vv-0922_0923-full_ST.tif vv-0922_0923-full_ST.tif

CPU times: user 220 µs, sys: 66 µs, total: 286 µs
Wall time: 169 µs


In [5]:
dsvh, dsvv = load_sen1(name_vh, name_vv)

Warning 1: TIFFReadDirectory:Invalid data type for tag StripByteCounts
Warning 1: TIFFFetchNormalTag:ASCII value for tag "GeoASCIIParams" contains null byte in value; value incorrectly truncated during reading due to implementation limitations
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Invalid data type for tag StripByteCounts
Warning 1: TIFFFetchNormalTag:ASCII value for tag "GeoASCIIParams" contains null byte in value; value incorrectly truncated during reading due to implementation limitations
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


LOAD SENTINEL 2



In [6]:
## cấu hình thời gian lấy ảnh và tọa độ
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.5, 106.4)
latitude_range = (9.2, 10.0) 

In [7]:
data = load_data(dc, date_range, longtitude_range, latitude_range)
notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

Most common native CRS: EPSG:32648
No datasets require offset correction
The valid_data_mask and scale (no offset) have been applied to the reflectance bands


<xarray.Dataset>
Dimensions:      (time: 151, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-02T03:35:23.960000 ... 2023-09...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [8]:
%%time
# Tiến hành loại bỏ các vị trí bị mây ảnh hưởng
result = mask_clean(data)
progress(result)

,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


{'0': 'no data',
 '1': 'saturated or defective',
 '2': 'dark area pixels',
 '3': 'cloud shadows',
 '4': 'vegetation',
 '5': 'bare soils',
 '6': 'water',
 '7': 'unclassified',
 '8': 'cloud medium probability',
 '9': 'cloud high probability',
 '10': 'thin cirrus',
 '11': 'snow or ice'}

CPU times: user 2.06 s, sys: 84.8 ms, total: 2.14 s
Wall time: 2.14 s


VBox()

CALCULATING THE MEAN VALUE AND FILL TO NAN POINT

In [9]:
ds1 = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = ds1["NDVI"]
average_ndvi = ndvi.resample(time='1M').mean().persist()  ## tính mean cho từng tháng -> time = 12
progress(average_ndvi)

VBox()

In [10]:
dsvh.shape

(13, 7680, 8687)

In [11]:
average_ndvi = average_ndvi.compute()
average_ndvi = average_ndvi[:, :dsvh.shape[1], :dsvh.shape[2]]

In [12]:
%%time
filled_ds = average_ndvi.bfill(dim='time')
filled_ds = filled_ds.ffill(dim='time')


CPU times: user 3.5 s, sys: 1.2 s, total: 4.7 s
Wall time: 4.69 s


FIND NAN POINT AFTER FILLING AND FILLING AGAIN WITH LINEARREGRESSION ALGORITHM

In [13]:
nan_mask = filled_ds.isnull()

# Print the NaN mask
# print(nan_mask)

# Count the number of NaNs
num_nans = nan_mask.sum()
print(f'Number of NaNs: {num_nans.values}')

Number of NaNs: 20800


In [14]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

mask = ~np.isnan(filled_ds)
X_train = np.stack([dsvh.values[mask], dsvv.values[mask]], axis=1)
y_train = filled_ds.values[mask]

In [15]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [16]:
X_pred = np.stack([dsvh.values[~mask], dsvv.values[~mask]], axis=1)
filled_ds.values[~mask] = model.predict(X_pred)

MATCH LABEL TO DATASET

In [17]:
%%time

# Takes 1 minute to complete.
loaded_datasets = {}
for idx, point in train.iterrows():
    key = f"point_{idx + 1}"
    try:
        ndvi_data = filled_ds.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
        vh_data = dsvh.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
        vv_data = dsvv.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
        loaded_datasets[key] = {
            "data": np.concatenate((ndvi_data, vh_data, vv_data)),
            "label": point.HT_code
                               }
    except Exception as e:
        # loaded_datasets[key] = None
        print(e)

CPU times: user 2.93 s, sys: 0 ns, total: 2.93 s
Wall time: 2.93 s


In [18]:
label_mapping = {
    "Lua tom": "0",
    "Lua": "1",
    "CHN": "2",
    "CLN": "3",
    "TS": "4",
    "Song": "5",
    "Dat xay dung": "6",
    "Rung": "7"
}
label_encoder = LabelEncoder()

# Fit and transform the labels
labels = train.Hientrang.values
numeric_labels = label_encoder.fit_transform([label_mapping[label] for label in labels])

In [19]:
X = []
x_new = []
lb_new = []
for k, v in loaded_datasets.items():
    X.append(v)
for i in range(len(X)):
    if X[i] is not None:
        x_new.append(X[i]["data"])
        lb_new.append(numeric_labels[i])

BUILDING DATASETS 

In [20]:
X_train, X_temp, y_train, y_temp= train_test_split(x_new, lb_new, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

TRAIN MODEL

In [21]:
%%time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the models
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
knn_model = KNeighborsClassifier()
nb_model = GaussianNB()
svm_model = SVC()

# Create a pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Apply scaling
    ('classifier', rf_model)       # Placeholder, will be set by param_grid
])

# Define the parameter grid for each classifier
param_grid = [
    # RandomForest
    {
        'classifier': [rf_model],
        'classifier__n_estimators': [100, 300, 500, 700],
        'classifier__max_depth': [6, 8, 10, 15],
        'classifier__criterion': ['gini', 'entropy'],
    },
    # KNeighborsClassifier
    {
        'classifier': [knn_model],
        'classifier__n_neighbors': [3, 5, 7, 9],
        'classifier__weights': ['uniform', 'distance'],
        'classifier__metric': ['euclidean', 'manhattan']
    },
    # Naive Bayes (GaussianNB doesn't have hyperparameters to tune here)
    {
        'classifier': [nb_model],
    },
    # SVM
    {
        'classifier': [svm_model],
        'classifier__C': [0.1, 1, 10, 100],
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__gamma': ['scale', 'auto']
    }
]

# Use GridSearchCV to find the best classifier and hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print out the best parameters and classifier
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Make predictions on the validation set
y_pred = grid_search.predict(X_val)

# Evaluate the results
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {round(accuracy, 2)*100} %")


Best Parameters: {'classifier': RandomForestClassifier(n_jobs=-1, random_state=42), 'classifier__criterion': 'gini', 'classifier__max_depth': 15, 'classifier__n_estimators': 300}
Accuracy: 86.0 %
CPU times: user 8.96 s, sys: 4.88 s, total: 13.8 s
Wall time: 26.8 s


In [22]:
## check accuracy score

y_pred_test = grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for test data {round(test_accuracy, 2)*100} %")

Accuracy for test data 86.0 %


In [23]:
dir_save_model = "model_train"
if not os.path.exists(dir_save_model):
    os.mkdir(dir_save_model)
joblib.dump(grid_search, os.path.join(dir_save_model, "model_new2.joblib"))

['model_train/model_new2.joblib']

In [24]:
client.close()
cluster.close()